In [3]:
pip install -U spacy

Requirement already up-to-date: spacy in /opt/conda/lib/python3.8/site-packages (2.3.5)
Note: you may need to restart the kernel to use updated packages.


In [4]:
!python3 -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [5]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [6]:
# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [7]:
# New label to add
LABEL = "FOOD"

# Training examples in the required format
TRAIN_DATA =[ ("Pizza is a common fast food.", {"entities": [(0, 5, "FOOD")]}),
              ("Pasta is an italian recipe", {"entities": [(0, 5, "FOOD")]}),
              ("China's noodles are very famous", {"entities": [(8,14, "FOOD")]}),
              ("Shrimps are famous in China too", {"entities": [(0,7, "FOOD")]}),
              ("Lasagna is another classic of Italy", {"entities": [(0,7, "FOOD")]}),
              ("Sushi is extemely famous and expensive Japanese dish", {"entities": [(0,5, "FOOD")]}),
              ("Unagi is a famous seafood of Japan", {"entities": [(0,5, "FOOD")]}),
              ("Tempura , Soba are other famous dishes of Japan", {"entities": [(0,7, "FOOD")]}),
              ("Udon is a healthy type of noodles", {"entities": [(0,4, "ORG")]}),
              ("Chocolate soufflé is extremely famous french cuisine", {"entities": [(0,17, "FOOD")]}),
              ("Flamiche is french pastry", {"entities": [(0,8, "FOOD")]}),
              ("Burgers are the most commonly consumed fastfood", {"entities": [(0,7, "FOOD")]}),
              ("Burgers are the most commonly consumed fastfood", {"entities": [(0,7, "FOOD")]}),
              ("Frenchfries are considered too oily", {"entities": [(0,11, "FOOD")]})
           ]

In [8]:
# Add the new label to ner
ner.add_label(LABEL)

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [9]:
# Importing requirements
from spacy.util import minibatch, compounding
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

  sizes = compounding(1.0, 4.0, 1.001)
  # Training for 30 iterations     
  for itn in range(30):
    # shuffle examples before training
    random.shuffle(TRAIN_DATA)
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=sizes)
    # ictionary to store losses
    losses = {}
    for batch in batches:
      texts, annotations = zip(*batch)
      # Calling update() over the iteration
      nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      print("Losses", losses)

/opt/conda/lib/python3.8/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "China's noodles are very famous" with entities "[(8, 14, 'FOOD')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)


Losses {'ner': 9.67839777469635}
Losses {'ner': 16.28438338706738}
Losses {'ner': 20.949838299066812}
Losses {'ner': 30.556954653706782}
Losses {'ner': 31.811866705411767}
Losses {'ner': 38.56815944667966}
Losses {'ner': 42.88868973846876}
Losses {'ner': 52.91209392011631}
Losses {'ner': 60.25382624328602}
Losses {'ner': 66.42045055970783}
Losses {'ner': 70.02622355604845}
Losses {'ner': 79.05023038133814}
Losses {'ner': 84.92190832067686}
Losses {'ner': 90.91724518109518}
Losses {'ner': 4.7757932296954095}
Losses {'ner': 8.288377240889155}
Losses {'ner': 18.575444773428522}
Losses {'ner': 22.222162102812188}
Losses {'ner': 25.462127272560945}
Losses {'ner': 33.04097256429411}
Losses {'ner': 37.237568848851126}
Losses {'ner': 43.21580649827699}
Losses {'ner': 49.694281383502286}
Losses {'ner': 57.65538201200722}
Losses {'ner': 60.657538434476166}
Losses {'ner': 62.64401739842576}
Losses {'ner': 65.67207889511626}
Losses {'ner': 69.89804507741216}
Losses {'ner': 2.611423085350907}
Losse

Losses {'ner': 12.030033373797778}
Losses {'ner': 13.168758162704762}
Losses {'ner': 17.871128900849726}
Losses {'ner': 25.41118572553387}
Losses {'ner': 25.699701531033497}
Losses {'ner': 28.459521697310265}
Losses {'ner': 33.37819176533958}
Losses {'ner': 36.39859484223416}
Losses {'ner': 43.03513793734601}
Losses {'ner': 49.502771071740426}
Losses {'ner': 53.668515345663764}
Losses {'ner': 53.95860195590649}
Losses {'ner': 7.996780771762133}
Losses {'ner': 11.321786031126976}
Losses {'ner': 14.445011986419559}
Losses {'ner': 17.40560843097046}
Losses {'ner': 20.127706850715185}
Losses {'ner': 21.918902170498768}
Losses {'ner': 26.497830624055496}
Losses {'ner': 28.313884731676808}
Losses {'ner': 30.36646957922494}
Losses {'ner': 38.44291324225924}
Losses {'ner': 41.97622516715637}
Losses {'ner': 42.05068601352832}
Losses {'ner': 47.61780159605405}
Losses {'ner': 51.75337329203467}
Losses {'ner': 3.9078431299421936}
Losses {'ner': 6.283941769666853}
Losses {'ner': 6.29728824346239}
L

In [10]:
# Testing the NER

test_text = "I ate Sushi yesterday. Maggi is a common fast food "
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
  print(ent)

Entities in 'I ate Sushi yesterday. Maggi is a common fast food '
Sushi
Maggi


In [12]:
# Output directory
from pathlib import Path
output_dir=Path('./content/')

# Saving the model to the output directory
if not output_dir.exists():
  output_dir.mkdir()
nlp.meta['name'] = 'my_ner'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

# Loading the model from the directory
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
assert nlp2.get_pipe("ner").move_names == move_names
doc2 = nlp2(' Dosa is an extremely famous south Indian dish')
for ent in doc2.ents:
  print(ent.label_, ent.text)

Saved model to content
Loading from content
FOOD Dosa
